In [ ]:
asdf = pd.DataFrame({'a':[2,4,6,8,10,12,14,16],
                     'b':[1,0,0,2,0,0,0,3]})
def zero_chunks(asdf, col_name):
    asdf['c'] = 1
    asdf.loc[asdf[col_name] == 0, 'c'] = asdf.loc[asdf[col_name] == 0, col_name]
    asdf['d'] = ~(asdf['c'].diff().eq(0) | asdf['c'].eq(0))
    asdf['e'] = asdf['d'].diff().eq(1).cumsum()


    # create new column with values from original index but starting at 0:
    asdf['f'] = asdf.index - asdf.index[0]

    # sort values based on 'd' and 'e' columns in ascending order:
    asdf.sort_values(by=['d', 'e'], ascending=True, inplace=True)

    # create empty column 'ind':
    asdf['ind'] = ''

    # assign values from 'f' column to 'ind' column where 'd' column is True: 
    asdf.loc[asdf['e'].diff().ne(0), 'ind'] = asdf.loc[asdf['e'].diff().ne(0), 'f']

    # create new column 'f2' with 1 shifted values from 'f' column:
    asdf['f2'] = asdf['f'].shift(1)

    # create new column 'ind2' and assign values from 'f2' column where 'ind' column is not empty:
    asdf['ind2'] = ''
    asdf.loc[asdf['ind'] != '', 'ind2'] = asdf.loc[asdf['ind'] != '', 'f2']

    # shift ind2 column by -1:
    asdf['ind2'] = asdf['ind2'].shift(-1)

    # create new column 'ind3' and assign values from 'ind' or 'ind2' columns where 'ind' or 'ind2'' columns are not empty:
    asdf['ind3'] = ''
    asdf.loc[asdf['ind'] != '', 'ind3'] = asdf.loc[asdf['ind'] != '', 'ind']
    asdf.loc[asdf['ind2'] != '', 'ind3'] = asdf.loc[asdf['ind2'] != '', 'ind2']


    # slice dataframe for 'False' values in 'd' column:
    asdf2 = asdf[asdf['d'] == False]

    # remove rows where 'ind3' column is empty and store it in 'asdf3' dataframe:
    asdf3 = asdf2[asdf2['ind3'] != '']

    # check asdf3:
    print(asdf3)

    # create 'start' column with values from 'ind3' column when 'ind2' is empty:
    asdf3['start'] = ''
    asdf3.loc[asdf3['ind2'] == '', 'start'] = asdf3.loc[asdf3['ind2'] == '', 'ind3']
    # create 'end' column with values from 'ind3' column when 'ind2' is not empty:
    asdf3['end'] = ''
    asdf3.loc[asdf3['ind2'] != '', 'end'] = asdf3.loc[asdf3['ind2'] != '', 'ind3']

    # shift 'end' column by -1:
    asdf3['end'] = asdf3['end'].shift(-1)

    # check asdf3:
    print(asdf3)

    # create new dataframe 'asdf4' with columns 'start' and 'end':
    asdf4 = asdf3[['start', 'end']]

    # remove None values from 'start' and 'end' columns:
    asdf4 = asdf4[asdf4['start'].notna() & asdf4['end'].notna()]

    # remove rows where 'start' and 'end' values are equal:
    asdf4 = asdf4[asdf4['start'] != asdf4['end']]

    # create 'count' column that counts from start (inclusive) to end (also inclusive):
    asdf4['count'] = asdf4['end'] - asdf4['start'] + 1

    # sort values in 'asdf4' dataframe by 'count' column in descending order:
    asdf4.sort_values(by='count', ascending=False, inplace=True)

    # check asdf4:
    return asdf4

print(zero_chunks(melted_data, 'KwH'))